# Taxi Example
Code obtained from [here](https://www.kaggle.com/charel/learn-by-example-reinforcement-learning-with-gym). 

Taxi has to pickup a passenger and then drop them off at a target location. Episode ends when the taxi has dropped off a passenger. Walls exist to prevent the taxi from moving. Colours indicate different objects/objectives.

### Rewards
* -1 for each action
* -10 for illegal action of pickup or dropoff
* +20 for a successful delivery

### Actions
* 0: move south
* 1: move north
* 2: move east 
* 3: move west 
* 4: pickup passenger
* 5: dropoff passenger

### Rendering 
* blue: passenger
* magenta: destination
* yellow: empty taxi
* green: full taxi
* other letters: locations



In [82]:
import gym 
import numpy as np 
import datetime
import keras 
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 

from time import sleep
from gym import envs
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
print("OK")

OK


Use command gym.make(ENVIRONMENT_NAME) to pick the Gym environment to use. Reset() initializes with random state from the state space. Render() prints the current environment state. Action space is the number of actions that the agent can take and their type. In Gym, the observation space indicates the state space as well as type.

In [83]:
env = gym.make('Taxi-v2')
print(env.action_space)
print("Actions : 0...%a" % str(env.action_space.n-1))
print(env.observation_space)

Discrete(6)
Actions : 0...'5'
Discrete(500)


Below is an example of taking an action and the environment changing as a result. Action_space.sample() chooses a random action. Step(ACTION) applies the action to the environment and returns teh new state, reward, boolean indicating trial completion and dictionary object for debugging.

In [84]:
rew_tot = 0
obs = env.reset()
print("Environment Start State")
env.render()
for _ in range(6):
    action = env.action_space.sample() 
    obs, rew, done, info = env.step(action) 
    rew_tot = rew_tot + rew
    env.render()
    
print("Reward: %r" % rew_tot) 

Environment Start State
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Reward: -24


## Value Iteration Algorithm
This approach calculates the value of each state based on taking an action in a state. The function "best_action_value" sets the environment to the parameter state and then applies each action, returning the action with the best value, which is determined using the Bellman Equation. 

In [85]:
def best_action_value(s):
    best_action = None
    best_value  = float('-inf')

    for a in range (0, NUM_ACTIONS):
        env.env.s = s
        s_new, reward, done, info = env.step(a) 
        v = reward + gamma * V[s_new]    
        
        if v > best_value:
            best_value = v
            best_action = a
    return best_action

V is an array that will hold the value of each state while $\pi$ is the policy, that is an array that will take in a state and return the best action. Gamma is the discount factor and the significant_improvement value is used as a stopping factor. 

In [86]:
NUM_ACTIONS = env.action_space.n
NUM_STATES = env.observation_space.n

V = np.zeros([NUM_STATES]) 
Pi = np.zeros([NUM_STATES], dtype=int)  
gamma = 0.9 
significant_improvement = 0.01 

### Training
Delta is used to calculate when to stop once it is smaller than the significant_improvement. Each iteration will calculate the value of each state (that is 500 states for the Taxi-V2 environment). Each state first has the best action selected, said best action is then taken and the environment updated, and finally the value is stored in the V array and $\pi$ array has the best action stored for that state. After this is done for each state, that iteration is over. On the next iteration, the V is calculated using the old_v (previous iteration's V).

In [87]:
iteration = 0

while True:
    delta = 0
    state = 0
    
    for s in range (0, NUM_STATES):
        old_v = V[s]
        action = best_action_value(s) 
        env.env.s = s  
        s_new, rew, done, info = env.step(action) 
        V[s] = rew + gamma * V[s_new] 
        Pi[s] = action
        delta = max(delta, np.abs(old_v - V[s]))
        state += 1
        
    iteration += 1
    if delta < significant_improvement:
        print (iteration,' iterations done')
        break

41  iterations done


### Testing
As the policy $\pi$ has now been obtained, it is used to map actions onto the states and thus allow the agent to complete the environment.

In [88]:
rew_tot = 0
obs  = env.reset()
print("Starting state")
env.render()
done = False

while done != True: 
    action = Pi[obs]
    obs, rew, done, info = env.step(action) #take step using selected action
    rew_tot = rew_tot + rew
    env.render()
    
print("Reward: %r" % rew_tot)  

Starting state
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
|_: : : : |
| | : | :